In [1]:
!pip install dmba

In [41]:
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import linear_model
import seaborn as sns
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge, LassoCV, BayesianRidge
from sklearn.feature_selection import SelectFromModel

# native python pkgs
from scipy import stats

# third-party python pkgs
from sklearn.utils import resample
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics


In [42]:
drive = pd.read_csv('drive_time_sedans.csv') # read csv file
drive.head(10)

,data_set,total_cost,lot_sale_days,overage,mileage,vehicle_type,domestic_import,vehicle_age,vehicle_age_group,color_set,makex,state,make_model
0,TRAIN,4037,135,YES,67341,FAMILY_LARGE,Domestic,8,SEVEN+,SILVER,BUICK,CA,BUICK_CENTURY
1,TRAIN,4662,18,NO,69384,FAMILY_SMALL,Import,4,FOUR,SILVER,MAZDA,TX,MAZDA_PROTEGE
2,TRAIN,4459,65,NO,58239,ECONOMY,Import,4,FOUR,RED,GEO,FL,GEO_METRO
3,TRAIN,4279,1,NO,58999,ECONOMY,Import,3,ONE-THREE,RED,KIA,TX,KIA_SEPHIA
4,TRAIN,4472,37,NO,47234,FAMILY_MEDIUM,Domestic,6,SIX,BLUE,MERCURY,NM,MERCURY_MYSTIQUE
5,TRAIN,5256,12,NO,64674,FAMILY_SMALL,Domestic,3,ONE-THREE,GOLD,CHEVROLET,CA,CHEVROLET_CAVALIER
6,TRAIN,5522,7,NO,92879,FAMILY_LARGE,Domestic,5,FIVE,RED,BUICK,NV,BUICK_LESABRE
7,TRAIN,3194,16,NO,67197,FAMILY_SMALL,Domestic,6,SIX,SILVER,PLYMOUTH,TX,PLYMOUTH_ACCLAIM
8,TRAIN,4332,7,NO,98403,FAMILY_MEDIUM,Domestic,4,FOUR,WHITE,CHEVROLET,NM,CHEVROLET_LUMINA
9,TRAIN,4249,42,NO,87183,FAMILY_MEDIUM,Domestic,6,SIX,SILVER,PONTIAC,FL,PONTIAC_GRAND AM


In [43]:
drive.describe()

,total_cost,lot_sale_days,mileage,vehicle_age
count,17506.000000,17506.000000,17506.000000,17506.000000
mean,4894.974009,46.819148,76950.447161,5.441049
std,934.798660,52.750454,16978.582173,1.751119
min,1375.000000,0.000000,82.000000,1.000000
25%,4300.000000,7.000000,67242.250000,4.000000
50%,4852.000000,24.000000,77961.000000,5.000000
75%,5444.000000,74.000000,89089.750000,7.000000
max,9813.000000,300.000000,119250.000000,12.000000


In [44]:
drive1 = (drive
     .drop(['data_set', 'overage', 'vehicle_type', 'domestic_import', 'vehicle_age_group', 'color_set', 'makex', 'state', 'make_model'], axis='columns'))

In [45]:
for col in drive1.columns: 
    print(col) 

total_cost
lot_sale_days
mileage
vehicle_age


In [71]:
independent = ['lot_sale_days', 'mileage', 'vehicle_age']
dependent = 'total_cost'

In [56]:
def mean_sq_prediction_error(y_actual, y_predicted):
    mpe = ((y_actual - y_predicted)**2).mean()
    print("Mean prediction error is % s" % round(mpe, 5)) 

In [47]:
# Split your data frame into training and test sets: 80% training; 20% test
X = pd.get_dummies(drive1[independent], drop_first=True)
y = drive1[dependent]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state=200)

In [68]:
#Running Logistic Regression 

result = smf.ols(formula='total_cost ~ lot_sale_days + mileage + vehicle_age', data=drive1).fit()    
print(result.summary()) 

                            OLS Regression Results                            
Dep. Variable:             total_cost   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     939.5
Date:                Tue, 19 Oct 2021   Prob (F-statistic):               0.00
Time:                        23:49:39   Log-Likelihood:            -1.4328e+05
No. Observations:               17506   AIC:                         2.866e+05
Df Residuals:                   17502   BIC:                         2.866e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      5631.3835     31.589    178.269

In [73]:
#Backwords Selection Method 

def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    return AIC_score(train_y, model.predict(train_X[variables]), model)

allVariables = train_X.columns
best_model, best_variables = backward_elimination(allVariables, train_model, score_model, verbose=True)

print(best_variables)

regressionSummary(test_y, best_model.predict(test_X[best_variables]))

Variables: lot_sale_days, mileage, vehicle_age
Start: score=229081.30
Step: score=229081.30, remove None
['lot_sale_days', 'mileage', 'vehicle_age']

Regression statistics

                      Mean Error (ME) : 5.4519
       Root Mean Squared Error (RMSE) : 887.3787
            Mean Absolute Error (MAE) : 673.9875
          Mean Percentage Error (MPE) : -3.5239
Mean Absolute Percentage Error (MAPE) : 14.6707


In [ ]:
#Decision Tree 

In [60]:
# Build the tree
tree_binary = tree.DecisionTreeClassifier()
tree_binary = tree_binary.fit(train_X, train_y)

# Show the tree if you like, following the code above

In [61]:
# Assign predicted values to an object
y_tree_predicted = tree_binary.predict(test_X)

In [62]:
# Convert comparison into a data frame and eyeball it
predictions = pd.DataFrame({'status': test_y,
                    'predicted_status': y_tree_predicted})

predictions

,status,predicted_status
9947,5362,5142
8462,4428,4790
1207,4878,4453
136,5917,5955
4458,5125,4600
...,...,...
6312,3905,4225
14963,5655,4044
12684,4162,6449
16701,5964,3738


In [63]:
# Compute the mean squared prediction error-- how well did the model perform?
mean_sq_prediction_error(predictions['status'], predictions['predicted_status'])

Mean prediction error is 1496073.13507
